---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [4]:
def date_sorter():

    global df

    # step 1
    new_df = df.copy()
    step1_regex = (
        r"(?P<fullstr>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))"
    )
    step1_df = new_df.str.extractall(step1_regex)
    # print("Number of matches: {}".format(step1_df.shape[0]))

    # Remove step 1 matches from df
    new_df = new_df[~new_df.index.isin(step1_df.index.get_level_values(0))]

    ######################

    # step 2
    step2_regex = r"(?P<fullstr>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))"
    step2_df = new_df.str.extractall(step2_regex)
    # print("Number of matches: {}".format(step2_df.shape[0]))

    # Remove step 2 matches from df
    new_df = new_df[~new_df.index.isin(step2_df.index.get_level_values(0))]
    all_data_df = step1_df.append(step2_df)
    ######################

    # step 3
    step3_regex = (
        r"(?P<fullstr>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))"
    )
    step3_df = new_df.str.extractall(step3_regex)
    # print("Number of matches: {}".format(step3_df.shape[0]))

    # Remove step 3 matches from df
    new_df = new_df[~new_df.index.isin(step3_df.index.get_level_values(0))]
    all_data_df = all_data_df.append(step3_df)
    ######################

    # step 4
    step4_regex = r"(?P<fullstr>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))"
    step4_df = new_df.str.extractall(step4_regex)
    # print("Number of matches: {}".format(step4_df.shape[0]))

    # Remove step 4 matches from df
    new_df = new_df[~new_df.index.isin(step4_df.index.get_level_values(0))]
    all_data_df = all_data_df.append(step4_df)
    ######################

    # step 5
    step5_regex = "(?P<fullstr>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))"
    step5_df = new_df.str.extractall(step5_regex)
    # print("Number of matches: {}".format(step5_df.shape[0]))

    # Remove step 5 matches from df
    new_df = new_df[~new_df.index.isin(step5_df.index.get_level_values(0))]
    all_data_df = all_data_df.append(step5_df)
    ######################

    # step 6
    step6_regex = r"(?P<fullstr>(?P<month>\d\d?)/(?P<year>\d{4}))"
    step6_df = new_df.str.extractall(step6_regex)
    # print("Number of matches: {}".format(step6_df.shape[0]))

    # Remove step 6 matches from df
    new_df = new_df[~new_df.index.isin(step6_df.index.get_level_values(0))]
    all_data_df = all_data_df.append(step6_df)
    ######################

    # step 7
    step7_regex = r"(?P<fullstr>(?P<year>\d{4}))"
    step7_df = new_df.str.extractall(step7_regex)
    # print("Number of matches: {}".format(step7_df.shape[0]))

    # Remove step 7 matches from df
    new_df = new_df[~new_df.index.isin(step7_df.index.get_level_values(0))]
    all_data_new = all_data_df.append(step7_df)

    ######################
    all_data = all_data_new
    # Handle missing strings
    all_data.loc[all_data["month"] == "Age", ["month", "day"]] = 1, 1

    all_data.loc[all_data["month"] == "Since", "month"] = 1

    # Append '19' to years with length == 2
    all_data.loc[all_data["year"].str.len() == 2, "year"] = (
        "19" + all_data.loc[all_data["year"].str.len() == 2, "year"]
    )

    # Fill missing days with 1
    all_data.loc[~all_data["day"].notnull(), "day"] = 1

    # Fill missing months with 1
    all_data.loc[~all_data["month"].notnull(), "month"] = 1

    # Reformat index
    all_data.reset_index(level="match", drop=True, inplace=True)
    all_data.drop([3, 4], axis=1, inplace=True)

    # Reorder columns
    all_data = all_data[["fullstr", "month", "day", "year"]]

    month_mapping = {
        "Apr": 4,
        "April": 4,
        "Aug": 8,
        "August": 8,
        "Dec": 9,
        "December": 9,
        "Decemeber": 9,
        "Feb": 2,
        "February": 2,
        "Jan": 1,
        "Janaury": 1,
        "January": 1,
        "Jul": 7,
        "July": 7,
        "Jun": 6,
        "June": 6,
        "Mar": 3,
        "March": 3,
        "May": 5,
        "Nov": 11,
        "November": 11,
        "Oct": 10,
        "October": 10,
        "Sep": 9,
        "September": 9,
    }

    all_data.replace({"month": month_mapping}, inplace=True)

    # create date str

    all_data["year_str"] = all_data.apply(
        lambda x: str(x["month"]) + "/" + str(x["day"]) + "/" + str(x["year"]), axis=1
    )

    all_data["datetime"] = all_data.apply(
        lambda x: pd.to_datetime(x["year_str"]), axis=1
    )

    sorted_order = pd.Series(all_data.sort_values(by="datetime").index)

    return sorted_order

In [6]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64